In [6]:

import numpy as np
import mrcfile
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib as mpl
import zarr
from numcodecs import Blosc
from scipy.ndimage import zoom

In [7]:
filenames = [
    "\\\\research.files.med.harvard.edu\\Neurobio\\GU LAB\\PERSONAL FILES\\Albert\\EEM004_mrc\\ec_big_0.mrc",
    "\\\\research.files.med.harvard.edu\\Neurobio\\GU LAB\\PERSONAL FILES\\Albert\\EEM004_mrc\\ec_big_1.mrc",
    "\\\\research.files.med.harvard.edu\\Neurobio\\GU LAB\\PERSONAL FILES\\Albert\\EEM004_mrc\\ec_big_2.mrc",
    "\\\\research.files.med.harvard.edu\\Neurobio\\GU LAB\\PERSONAL FILES\\Albert\\EEM004_mrc\\ec_big_3.mrc",
]

In [8]:
compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.BITSHUFFLE)

for i, filename in enumerate(filenames):
    mrc = mrcfile.mmap(filename, mode='r+')
    z = zarr.array(np.array(mrc.data, dtype='uint8'), chunks=(64, 128, 128), compressor=compressor)


    s0 = np.array(mrc.data, dtype='uint8')
    s1 = zoom(s0, 0.5)
    s2 = zoom(s1, 0.5)
    s3 = zoom(s2, 0.5)
    s4 = zoom(s3, 0.5)
    s5 = zoom(s4, 0.5)
    scales = [s0, s1, s2, s3, s4, s5]

    store = zarr.DirectoryStore(f'\\\\research.files.med.harvard.edu\\Neurobio\\GU LAB\\PERSONAL FILES\\Albert\\EEM004_mrc\\ec_big_{i}.zarr')
    root = zarr.group(store=store, overwrite=True)
    fibsem_group = root.create_group('fibsem-uint8')


    fibsem_group.attrs['axes'] = ['z', 'y', 'x']
    fibsem_group.attrs['units'] = ['nm', 'nm', 'nm']
    fibsem_group.attrs['pixelResolution'] = {'dimensions': [40, 40, 40], 'unit': 'nm'}
    fibsem_group.attrs['scales'] = [[1, 1, 1], [2, 2, 2], [4, 4, 4], [8, 8, 8], [16, 16, 16], [32, 32, 32]]

    dim_scales = [8, 16, 32, 64, 128, 256]
    dims = [list(s.shape) for s in scales]

    datasets = []
    for j in range(0, len(dim_scales)):
        datasets.append({
            "path": f"s{j}",
            "coordinateTransformations": [{
                "type": "scale",
                "scale": [float(dim_scales[j]), float(dim_scales[j]), float(dim_scales[j])]
            }],
            "compression": {
                'type': 'blosc',
                'cname': 'lz4',
                'clevel': 5,
                'shuffle': 'bit',
            },
            "blockSize": [64, 128, 128],
            "dataType": "uint8",
            "dimensions": dims[j]
        })

    fibsem_group.attrs['multiscales'] = [{
        "version": "0.4",
        "name": "fibsem-uint8",
        "axes": [
            # {"name": "t", "type": "time", "unit": "millisecond"},
            # {"name": "c", "type": "channel"},
            {"name": "z", "type": "space", "unit": "nanometer"},
            {"name": "y", "type": "space", "unit": "nanometer"},
            {"name": "x", "type": "space", "unit": "nanometer"}
        ],
        'datasets': datasets,
        "coordinateTransformations": [{
            "type": "scale",
            "scale": [1.0, 1.0, 1.0]
        }],
        "type": "gaussian",
    }]

    
    for j, scale in enumerate(scales):
        z = fibsem_group.array(f's{j}', scale, chunks=(64, 128, 128), compressor=compressor)
    
    store.close()